![DLI Header](images/DLI_Header.png)

# 星辰自行车营销文案生成器

在这个笔记本中，您将构建一个 AI 驱动的营销文案撰写器，能够执行多种生成任务。您将学习如何编辑模型的**系统消息** 来定义其响应生成的职责。

## 学习目标

在完成此笔记本时，您将能够：
- 使用 LLaMA-2 执行各种**文本生成** 任务。
- 使用**系统上下文** 为 LLaMA-2 模型提供其总体职责的定义。

## 视频演练

执行下面的单元格以加载此笔记本的视频演练。

In [ ]:
from IPython.display import HTML

video_url = "https://d36m44n9vdbmda.cloudfront.net/assets/s-fx-12-v1/v2/04-copy.mp4"

video_html = f"""
<video controls width="640" height="360">
    <source src="{video_url}" type="video/mp4">
    Your browser does not support the video tag.
</video>
"""

display(HTML(video_html))

## 创建 LLaMA-2 管道

In [ ]:
from transformers import pipeline
model = "TheBloke/Llama-2-13B-chat-GPTQ"
# model = "TheBloke/Llama-2-7B-chat-GPTQ"

llama_pipe = pipeline("text-generation", model=model, device_map="auto");

## 辅助函数

在此笔记本中，我们将使用以下函数来支持我们与 LLM 的交互。现在可以随意浏览一下，因为在下面使用时会更详细地介绍。

### 生成模型响应

In [ ]:
def generate(prompt, max_length=1024, pipe=llama_pipe, **kwargs):
    """
    Generates a response to the given prompt using a specified language model pipeline.

    This function takes a prompt and passes it to a language model pipeline, such as LLaMA, 
    to generate a text response. The function is designed to allow customization of the 
    generation process through various parameters and keyword arguments.

    Parameters:
    - prompt (str): The input text prompt to generate a response for.
    - max_length (int): The maximum length of the generated response. Default is 1024 tokens.
    - pipe (callable): The language model pipeline function used for generation. Default is llama_pipe.
    - **kwargs: Additional keyword arguments that are passed to the pipeline function.

    Returns:
    - str: The generated text response from the model, trimmed of leading and trailing whitespace.

    Example usage:
    ```
    prompt_text = "Explain the theory of relativity."
    response = generate(prompt_text, max_length=512, pipe=my_custom_pipeline, temperature=0.7)
    print(response)
    ```
    """

    def_kwargs = dict(return_full_text=False, return_dict=False)
    response = pipe(prompt.strip(), max_length=max_length, **kwargs, **def_kwargs)
    return response[0]['generated_text'].strip()

### 构造提示，可选地使用系统上下文和/或示例
｜Costruct Prompt, Optionally With System Context and/or Examples

In [ ]:
def construct_prompt_with_context(main_prompt, system_context="", conversation_examples=[]):
    """
    Constructs a complete structured prompt for a language model, including optional system context and conversation examples.

    This function compiles a prompt that can be directly used for generating responses from a language model. 
    It creates a structured format that begins with an optional system context message, appends a series of conversational 
    examples as prior interactions, and ends with the main user prompt. If no system context or conversation examples are provided,
    it will return only the main prompt.

    Parameters:
    - main_prompt (str): The core question or statement for the language model to respond to.
    - system_context (str, optional): Additional context or information about the scenario or environment. Defaults to an empty string.
    - conversation_examples (list of tuples, optional): Prior exchanges provided as context, where each tuple contains a user message 
      and a corresponding agent response. Defaults to an empty list.

    Returns:
    - str: A string formatted as a complete prompt ready for language model input. If no system context or examples are provided, returns the main prompt.

    Example usage:
    ```
    main_prompt = "I'm looking to improve my dialogue writing skills for my next short story. Any suggestions?"
    system_context = "User is an aspiring author seeking to enhance dialogue writing techniques."
    conversation_examples = [
        ("How can dialogue contribute to character development?", "Dialogue should reveal character traits and show personal growth over the story arc."),
        ("What are some common pitfalls in writing dialogue?", "Avoid exposition dumps in dialogue and make sure each character's voice is distinct.")
    ]

    full_prompt = construct_prompt_with_context(main_prompt, system_context, conversation_examples)
    print(full_prompt)
    ```
    """
    
    # Return the main prompt if no system context or conversation examples are provided
    if not system_context and not conversation_examples:
        return main_prompt

    # Start with the initial part of the prompt including the system context, if provided
    full_prompt = f"<s>[INST] <<SYS>>{system_context}<</SYS>>\n" if system_context else "<s>[INST]\n"

    # Add each example from the conversation_examples to the prompt
    for user_msg, agent_response in conversation_examples:
        full_prompt += f"{user_msg} [/INST] {agent_response} </s><s>[INST]"

    # Add the main user prompt at the end
    full_prompt += f"{main_prompt} [/INST]"

    return full_prompt

## 星辰自行车数据

In [ ]:
bikes = [
    {
        "model": "Galaxy Rider",
        "type": "Mountain",
        "features": {
            "frame": "Aluminum alloy",
            "gears": "21-speed Shimano",
            "brakes": "Hydraulic disc",
            "tires": "27.5-inch all-terrain",
            "suspension": "Full, adjustable",
            "color": "Matte black with green accents"
        },
        "usps": ["Lightweight frame", "Quick gear shift", "Durable tires"],
        "price": 799.95,
        "internal_id": "GR2321",
        "weight": "15.3 kg",
        "manufacturer_location": "Taiwan"
    },
    {
        "model": "Nebula Navigator",
        "type": "Hybrid",
        "features": {
            "frame": "Carbon fiber",
            "gears": "18-speed Nexus",
            "brakes": "Mechanical disc",
            "tires": "26-inch city slick",
            "suspension": "Front only",
            "color": "Glossy white"
        },
        "usps": ["Sleek design", "Efficient on both roads and trails", "Ultra-lightweight"],
        "price": 649.99,
        "internal_id": "NN4120",
        "weight": "13.5 kg",
        "manufacturer_location": "Germany"
    },
    {
        "model": "Cosmic Comet",
        "type": "Road",
        "features": {
            "frame": "Titanium",
            "gears": "24-speed Campagnolo",
            "brakes": "Rim brakes",
            "tires": "700C road",
            "suspension": "None",
            "color": "Metallic blue"
        },
        "usps": ["Super aerodynamic", "High-speed performance", "Professional-grade components"],
        "price": 1199.50,
        "internal_id": "CC5678",
        "weight": "11 kg",
        "manufacturer_location": "Italy"
    }
]

## 完整的 LLaMA-2 提示模板
｜The Full LLaMA-2 Prompt Template

在之前的笔记本中，我们利用 LLaMA-2 的**提示模板（prompt template）** 来支持**少样本学习（few-shot learning）** ，但提到我们正在使用略微修改版本的提示模板。具体来说，我们省略了提示模板的用户消息部分，称为**系统消息（system message）** 、**系统上下文（system context）** 或**系统提示（system prompt）** (我们将交替使用这些术语)。以下是完整的 LLaMA-2 提示模板，包括其**系统上下文** 部分。

```python
<s>[INST] <<SYS>>
{{ system_context }}
<</SYS>>

{{ user_msg_1 }} [/INST] {{ model_answer_1 }} </s>

```

**系统上下文** 是用户/模型交互的用户端的一部分，位于`<<SYS>>`和`<</SYS>>`标签之间。**系统上下文** 是一个初步的陈述或上下文提示，旨在将 AI 模型的响应引导到特定框架或对任务的理解。

关于什么属于**系统上下文** 没有硬性规定，但我们应该主要将其视为设置模型的职责，或任何将适用于其所有响应的上下文。

以下是 LLaMA-2 聊天模型的默认**系统消息** ，在其指令微调期间使用：

>您是一位乐于助人、尊重他人和诚实的助手。始终尽可能提供帮助，同时保持安全。您的答案不应包含任何有害、不道德、种族主义、性别歧视、有毒、危险或非法的內容。请确保您的回复在社会上公正且积极。
>如果问题毫无意义或在事实上一致性，请解释原因，而不是回答错误的东西。如果您不知道某个问题的答案，请不要分享虚假信息。

>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
>If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

## 设置系统上下文
｜Setting the System Context

以下`construct_prompt_with_context`函数将帮助我们使用 LLaMA-2**提示模板** 构建具有更新**系统消息** 的提示。此函数还允许我们，如果我们愿意，通过传入 2 元组示例交互列表来执行**少样本学习** ，就像我们在上一节中所做的那样。

In [ ]:
def construct_prompt_with_context(main_prompt, system_context="", conversation_examples=[]):
    """
    Constructs a complete structured prompt for a language model, including optional system context and conversation examples.

    This function compiles a prompt that can be directly used for generating responses from a language model. 
    It creates a structured format that begins with an optional system context message, appends a series of conversational 
    examples as prior interactions, and ends with the main user prompt. If no system context or conversation examples are provided,
    it will return only the main prompt.

    Parameters:
    - main_prompt (str): The core question or statement for the language model to respond to.
    - system_context (str, optional): Additional context or information about the scenario or environment. Defaults to an empty string.
    - conversation_examples (list of tuples, optional): Prior exchanges provided as context, where each tuple contains a user message 
      and a corresponding agent response. Defaults to an empty list.

    Returns:
    - str: A string formatted as a complete prompt ready for language model input. If no system context or examples are provided, returns the main prompt.

    Example usage:
    ```
    main_prompt = "I'm looking to improve my dialogue writing skills for my next short story. Any suggestions?"
    system_context = "User is an aspiring author seeking to enhance dialogue writing techniques."
    conversation_examples = [
        ("How can dialogue contribute to character development?", "Dialogue should reveal character traits and show personal growth over the story arc."),
        ("What are some common pitfalls in writing dialogue?", "Avoid exposition dumps in dialogue and make sure each character's voice is distinct.")
    ]

    full_prompt = construct_prompt_with_context(main_prompt, system_context, conversation_examples)
    print(full_prompt)
    ```
    """
    
    # Return the main prompt if no system context or conversation examples are provided
    if not system_context and not conversation_examples:
        return main_prompt

    # Start with the initial part of the prompt including the system context, if provided
    full_prompt = f"<s>[INST] <<SYS>>{system_context}<</SYS>>\n" if system_context else "<s>[INST]\n"

    # Add each example from the conversation_examples to the prompt
    for user_msg, agent_response in conversation_examples:
        full_prompt += f"{user_msg} [/INST] {agent_response} </s><s>[INST]"

    # Add the main user prompt at the end
    full_prompt += f"{main_prompt} [/INST]"

    return full_prompt

## 星辰自行车营销文案生成器

让我们使用我们的 LLaMA-2 模型作为营销文案生成器。对于其任务，我们将提供一个 JSON 对象，其中包含有关我们想要为其撰写文案的星辰自行车模型的相关规格。如果您还没有，请查看上面的“星辰自行车数据”部分以获取`bikes`的定义。

我们将从一个简单的提示开始。

In [ ]:
prompt = f"""
Write marketing copy for the following bicycle: {bikes[0]}
"""

print(generate(prompt))

---

这还不错，并且根据您已经了解的内容，您可能会迭代一个提示来改进模型的响应。但是假设我们希望我们的模型充当营销文案撰写器，可能以各种格式撰写文案，让我们通过提供有关其职责的知识来提供**系统上下文** 。

In [ ]:
system_context = f"""
You are a marketing copy writer for Star Bikes.
"""

prompt = f"""
{bikes[0]}
"""

使用我们的`construct_prompt_with_context`函数，我们现在可以创建一个提示，该提示为 LLaMA-2 提示模板设置适当的**系统上下文** 。

In [ ]:
prompt_with_system_context = construct_prompt_with_context(prompt, system_context)
print(prompt_with_system_context)

---

使用我们带有**系统上下文** 的提示，让我们看看我们从模型中得到了什么样的响应。值得一提的是，我们的主要`prompt`（见上文）没有提供任何关于模型需要做什么的说明。我们依靠设置的**系统上下文** 来指导模型的行为。

In [ ]:
print(generate(prompt_with_system_context))

---

这还不错，但让我们在**系统上下文** 中更**精确** 地说明模型的职责仅限于生成营销文案，而不是像我们在上一个响应中获得的那样生成任何引导性对话文本，例如`当然！这是 Galaxy Rider 山地自行车的营销文案：`。

In [ ]:
system_context = f"""
You are a marketing copy writer for Star Bikes. You only write marketing copy and never any \
leading comments or pieces of conversation.
"""

prompt = f"""
{bikes[0]}
"""

print(generate(construct_prompt_with_context(prompt, system_context)))

---

这似乎没有帮助。让我们进行迭代。也许如果我们告诉模型它是一台机器，它就不会试图进行类似人类的对话。

In [ ]:
system_context = f"""
You are a non-conversant machine that generates marketing copy in 100 words or less. You work for Star Bikes.
"""

prompt = f"""
{bikes[0]}
"""

print(generate(construct_prompt_with_context(prompt, system_context)))

---

好多了！就像所有提示工程一样，开发有效的**系统提示** 通常是一个迭代过程。

## 强制简洁
｜Enforce Brevity

让我们假设我们只希望获得约 100 个单词的响应。我们可以更新**系统上下文** 以反映这一点。

In [ ]:
system_context = f"""
You are a non-conversant machine that generates marketing copy in 100 words or less. You work for Star Bikes.
"""

prompt = f"""
{bikes[0]}
"""

print(generate(construct_prompt_with_context(prompt, system_context)))

---

太好了。现在我们已经建立了一个似乎对我们有效的设置，让我们在其他自行车的數據上试一试：

In [ ]:
for bike in bikes[1:]:
    print(generate(construct_prompt_with_context(bike, system_context)))
    print("\n-----\n")

## 练习：生成营销邮件

利用您到目前为止学到的知识，创建一个提示（可能利用其**系统上下文** ）为特定自行车的用户撰写营销邮件。电子邮件应以收件人的姓名称呼收件人。

如果您遇到困难，请查看下面的解决方案。

### 您的工作在此处

### 解决方案

In [ ]:
system_context = f"""
You are a non-conversant machine that generates marketing emails in 100 words or less. You work for Star Bikes.
"""

prompt = f"""
Recipient Name: Stella
{bikes[0]}
"""

print(generate(construct_prompt_with_context(prompt, system_context)))

## 关键概念回顾

本笔记本中介绍了以下关键概念：

- **系统消息 System Message：** 指令微调模型（fine-tuned models）的提示模板（prompt templates）的一部分，允许用户设置其行为的职责（role）或总体上下文（overarching context）。

## 可选的高级练习

如果您想超越课程的要求，以下是一些供您尝试的额外开放式练习。

### 使用 7B 模型

在笔记本的顶部，在重新启动内核（请参见下面的单元格）之后，取消注释并使用 7B 模型而不是我们演示的 13B 模型。尝试在使用较小（较弱）模型的情况下获得令人满意的结果。

### 实验以找到最佳方法

我们通过结合基本提示的迭代、系统消息的编辑以及提供示例（又称“样本”，shots）来帮助模型，得出了几个有效的提示。哪种方法效果更好，通常更像是一种艺术而不是科学：看看您是否可以通过强调所有这三个“杠杆（levers）”的变化来获得可接受的结果。

### 创建电子邮件生成管道

扩展上面练习中的工作，创建一个管道，给定一组收件人，可以为他们每个人生成电子邮件。您可以考虑创建更多内容的合成用户数据，而不仅仅是收件人的姓名，例如他们之前购买或表示感兴趣的自行车、是否已向他们发送过电子邮件等，然后生成与这些详细信息更相关的电子邮件。

您还可以考虑在**少样本（few-shot）** 学习的背景下做更多的事情，以特定方式构建电子邮件。

## 重新启动内核
｜Restart the Kernel

为了释放下一个笔记本的 GPU 内存，请运行以下单元格以重新启动内核。

In [ ]:
from IPython import get_ipython

get_ipython().kernel.do_shutdown(restart=True)

# [5LOI DEEP LEARNING INSTITUE](https://5loi.com)

### [AI COMMUNITY](https://www.theforage.cn/community)

### [5LOI](https://5loi.com/about_loi)

![DLI Header](images/DLI_Header.png)